In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pickle
import pandas as pd
from simulation import WangStamatiadis
from scipy.stats import ks_2samp

## Wang Stamatiadis TTC measure

In [ ]:
ttc = np.linspace(0.01, 4, 100)
ws = WangStamatiadis()
pcollision = ws.groupa(ttc)

In [ ]:
plt.plot(ttc, pcollision)
plt.xlabel("TTC [s]")
plt.ylabel("Probability of collision")
plt.title("Group A")
plt.xlim(0, np.max(ttc))
plt.show()

In [ ]:
ttc = np.linspace(0.01, 4, 100)
vdiff = np.linspace(0, 30, 50)
groupb2 = np.zeros((len(ttc), len(vdiff)))
pcollision = np.zeros_like(groupb2)
for i, t in enumerate(ttc):
    for j, v in enumerate(vdiff):
        groupb2[i, j] = ws.groupb2(t, v)
        pcollision[i, j] = ws.prob_collision(t, v)

In [ ]:
f, ax = plt.subplots()
cf = ax.contourf(ttc, vdiff, groupb2.T, levels=np.linspace(0, 1, 11))
plt.xlabel("TTC [s]")
plt.ylabel("Speed difference [m/s]")
plt.title("Group B2")
f.colorbar(cf)

In [ ]:
f, ax = plt.subplots()
cf = ax.contourf(ttc, vdiff, pcollision.T, levels=np.linspace(0, 1, 11))
plt.xlabel("TTC [s]")
plt.ylabel("Speed difference [m/s]")
plt.title("Group A + Group B2")
f.colorbar(cf)

## Comparison of metrics

In [ ]:
# Load data.
filename = os.path.join("data", "8_interactions_v3", "interactions_wi10.pkl")
with open(filename, 'rb') as file:
    all_interactions = pickle.load(file)
locations = sorted(all_interactions.keys())

In [ ]:
columns = ['TTC', 'CPM_TTC', 'CPM_WS', 'prob_collision', 'Anomaly_Reconstruction',
           'TTC2', 'TTC2_filtered', 'CPM_TTC2', 'CPM_TTC2_filtered', 'CPM_WS2',
           'CPM_WS2_filtered']
scores = []
for location in locations:
    for interaction in all_interactions[location].values():
        if "scores" not in interaction:
            continue
        if "prob_collision" not in interaction['scores']:
            continue
        scores.append(interaction['scores'][columns])
df_scores = pd.DataFrame(np.concatenate(scores), columns=columns)

## Percentiles

In [ ]:
def plot_percentile(p):
    plt.plot(p, np.percentile(df_scores["CPM_TTC2"], p), label="CPM_TTC")
    plt.plot(p, np.percentile(df_scores["CPM_WS2"], p), label="CPM_WS")
    plt.plot(p, np.percentile(df_scores["prob_collision"], p), label="Erwin")
    plt.legend()
    plt.xlabel("Percentile")
    plt.ylabel("Probability of collision")
plot_percentile(np.linspace(0, 100, 101))

In [ ]:
plot_percentile(np.linspace(99, 100, 101))
plt.ylim(-0.01, 1)

In [ ]:
plot_percentile(np.linspace(99.9, 100, 101))
plt.ylim(-0.01, 1)

In [ ]:
def plot_percentile2(p):
    plt.plot(p, np.percentile(df_scores["Anomaly_Reconstruction"], p))
    plt.xlabel("Percentile")
    plt.ylabel("Anomaly reconstruction")
plot_percentile2(np.linspace(0, 100, 101))

In [ ]:
plot_percentile2(np.linspace(99, 100, 101))

## Correlation

In [ ]:
plt.plot(df_scores["CPM_WS2"], df_scores["prob_collision"], '.')
plt.xlabel("Wang Stamatiadis metric")
plt.ylabel("Erwin's metric")
plt.title("Correlation: {:.4f}".format(np.corrcoef(df_scores["CPM_WS2"], 
                                                   df_scores["prob_collision"])[0][1]))

In [ ]:
plt.plot(df_scores["CPM_WS2"], df_scores["Anomaly_Reconstruction"], '.')
plt.xlabel("Wang Stamatiadis metric")
plt.ylabel("Anomaly reconstruction error")
plt.title("Correlation: {:.4f}".format(np.corrcoef(df_scores["CPM_WS2"], 
                                                   df_scores["Anomaly_Reconstruction"])[0][1]))

In [ ]:
plt.plot(df_scores["CPM_WS2"], df_scores["Anomaly_Reconstruction"], '.')
plt.xlabel("Wang Stamatiadis metric")
plt.ylabel("Anomaly reconstruction error")
plt.title("Correlation: {:.4f}".format(np.corrcoef(df_scores["CPM_WS2"], 
                                                   df_scores["Anomaly_Reconstruction"])[0][1]))
plt.ylim(0, 10)

In [ ]:
plt.plot(df_scores["prob_collision"], df_scores["Anomaly_Reconstruction"], '.')
plt.xlabel("Erwin's metric")
plt.ylabel("Anomaly reconstruction error")
plt.title("Correlation: {:.4f}".format(np.corrcoef(df_scores["CPM_WS2"], 
                                                   df_scores["Anomaly_Reconstruction"])[0][1]))